# Evaluation

# Setup

In [1]:
import pandas as pd
from pathlib import Path
import os
import sys
import json
import time
from datetime import datetime
from dotenv import load_dotenv
import warnings
warnings.filterwarnings("ignore")
from sentence_transformers import SentenceTransformer

project_root = os.path.abspath('..')
if project_root not in sys.path:
    sys.path.insert(0, project_root)
from utils import get_table_from_supabase, build_relationship_cols, build_financial_history, extract_classifications
from evaluation_utils import get_recipients_by_id

#get keys from env
load_dotenv()
url = os.getenv("SUPABASE_URL")
key = os.getenv("SUPABASE_KEY")

----

# Retrieving Data from Checkpoints

In [2]:
#get checkpoint folder
checkpoint_folder = Path("./13.1_checkpoints/")

#get checkpoint
funders_df = pd.read_pickle(checkpoint_folder / "funders_df.pkl")
grants_df = pd.read_pickle(checkpoint_folder / "grants_df.pkl")
areas_df = pd.read_pickle(checkpoint_folder / "areas_df.pkl")
hierarchies_df = pd.read_pickle(checkpoint_folder / "hierarchies_df.pkl")
eval_df = pd.read_pickle(checkpoint_folder / "eval_df.pkl")

# Retrieving Data from Supabase and Building Dataframes

In [3]:
#get data for evaluation recipients
recipient_ids = eval_df["recipient_id"].unique()
recipients_df = get_recipients_by_id(url, key, recipient_ids)

In [4]:
recipients_df

,recipient_id,recipient_name,recipient_activities,is_recipient,recipient_objectives,recipient_name_em,recipient_activities_em,recipient_objectives_em,recipient_concat_em,recipient_extracted_class,recipient_areas,recipient_causes,recipient_beneficiaries
0,1000340,FREEDOM FROM TORTURE,FREEDOM FROM TORTURE IS A HUMAN RIGHTS ORGANIS...,True,3.1 THE OBJECTS OF THE CHARITY ARE FOR THE PUB...,"[0.0014025384,0.060177933,-0.008869013,0.01545...","[0.0012441295,0.03081011,-0.012429256,-0.01267...","[-0.0042924527,0.05366534,-0.030244896,-0.0456...","[-0.018466309,0.035319023,-0.006930157,-0.0269...","[""UK"",""ASYLUM SEEKERS AND REFUGEES"",""FAMILIES""...","[Scotland, Europe, Manchester City, Greater Ma...","[The Advancement Of Health Or Saving Of Lives,...",[Other Defined Groups]
1,1000414,WELDMAR HOSPICECARE,PROVISION OF SPECIALIST PALLIATIVE CARE TO ADU...,True,TO PROMOTE THE RELIEF OF SICKNESS BY SUCH CHAR...,"[-0.046828583,0.047270894,-0.074979275,0.00607...","[-0.040439986,0.0126432255,-0.059748463,-0.003...","[-0.053749353,0.02162124,-0.04952093,-0.018352...","[-0.056574777,0.01436219,-0.0409797,-0.0121981...","[""SOUTH"",""WEST OF DORSET"",""DORCHESTER"",""ENGLAN...",[Dorset],[The Advancement Of Health Or Saving Of Lives],"[Elderly/old People, Other Defined Groups, The..."
2,1112590,CANTERBURY CATHEDRAL TRUST FUND,THE TRUST SUPPORTS THE PRIORITIES OF CANTERBUR...,True,"3.1 THE CONSERVATION, MAINTENANCE AND IMPROVEM...","[0.0312815,0.013435593,-0.034705196,-0.0178996...","[-0.011534776,0.013584374,-0.012827868,-0.0290...","[-0.013078383,0.030979622,-0.0064046737,-0.054...","[-0.03383201,-0.01068588,0.0027609572,-0.03315...","[""CHOIRS"",""MUSIC"",""WOMEN"",""FUNDRAISING"",""HERIT...",[Kent],"[Education/training, Religious Activities, Art...","[Children/young People, Other Charities Or Vol..."
3,1146676,ANIMAL RESCUE CYMRU,"SMALL ANIMAL CHARITY, TAKING IN AND RESCUING A...",False,FOR THE BENEFIT OF THE PUBLIC IN WALES TO RELI...,None,None,None,None,None,[Throughout Wales],[Animals],[The General Public/mankind]
4,1162478,WILD TROUT TRUST LIMITED,THE ADVANCEMENT OF PUBLIC EDUCATION IN THE PRO...,True,"THE CHARITY'S OBJECTS (""OBJECTS"") ARE SPECIFIC...","[-0.01954093,0.044843018,-0.019942464,0.013304...","[-0.01661305,0.03053053,-0.015281906,-0.011168...","[-0.04861267,0.03547574,-0.03891287,-0.0156953...","[-0.08538764,0.01833849,-0.035040997,0.0001340...","[""POLICY CAMPAIGNING AND ADVOCACY"",""WATER"",""ED...","[Ireland, Europe, Scotland, Throughout England...","[Education/training, Environment/conservation/...","[Other Charities Or Voluntary Bodies, The Gene..."
5,1186661,FAITH IN LATER LIFE LTD,PROVIDING RESOURCES AND INFORMATION REGARDING ...,True,TO ADVANCE THE CHRISTIAN FAITH BY INSPIRING AN...,"[0.047914438,0.06354407,-0.026701199,-0.018678...","[0.029653521,0.031717736,-0.03671217,0.0374967...","[0.035304695,0.050040435,-0.04534686,-0.006704...","[0.029496266,0.036098488,-0.0327703,0.03646846...","[""ASSOCIATIONS"",""OLDER PEOPLE"",""CHRISTIANITY"",...","[Scotland, Europe, Northern Ireland, Throughou...",[Religious Activities],[Elderly/old People]
6,1194238,KIDZ KLUB - LEEDS,KIDZ KLUB LEEDS EXISTS TO PROMOTE AND WORK TO ...,True,THE OJECTS OF THE CIO ARE:(1) TO ACT AS A RESO...,"[-0.0018421615,0.0014972656,-0.011559817,-0.00...","[-0.011810082,-0.018335436,-0.019618865,-0.014...","[0.01757876,0.046465892,-0.046633597,-0.029315...","[-0.01104823,0.009532384,-0.0026252083,-0.0241...","[""LEEDS"",""THE UNITED KINGDOM"",""WALES"",""CHILDRE...","[Leeds City, West Yorkshire]","[Education/training, Religious Activities, Eco...",[Children/young People]
7,1201946,SEVENTH HEAVEN,THE CHARITY RUNS EVENTS FOR YOUNG PEOPLE WITH ...,False,"TO ADVANCE IN LIFE AND HELP YOUNG PEOPLE, ESPE...",None,None,None,None,None,"[Hackney, Haringey]","[Disability, Other Charitable Purposes]","[Children/young People, People With Disabilities]"
8,1205530,NORTHERN CANCER VOICES,NORTHERN CANCER VOICES ARE FORMED FROM A GROUP...,False,"FOR THE PUBLIC BENEFI